In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Postgres_to_Landing").getOrCreate()

landing_path = "/Volumes/movies_catalog/landing/landing_zone_movies"

dbutils.fs.mkdirs(landing_path)

host = "postgres.c0fk60kqcz2v.us-east-1.rds.amazonaws.com"
porta = "5432"
database = "postgres"
usuario = "postgres"
senha = "postgres"

jdbc_url = f"jdbc:postgresql://{host}:{porta}/{database}"

connection_properties = {
    "user": usuario,
    "password": senha,
    "driver": "org.postgresql.Driver"
}

tables = [
    "movies_top100",
    "movies_casts",
    "movies_meta",
    "movies_ratings",
    "movies_revenue"
]

for table in tables:
    print(f"📥 Lendo tabela: {table}")

    df = spark.read.jdbc(
        url=jdbc_url,
        table=f"public.{table}",
        properties=connection_properties
    )

    output_path = f"{landing_path}/{table}.csv"

    df.write.mode("overwrite").option("header", True).csv(output_path)

    print(f"✅ Tabela {table} carregada para Landing → {output_path}")
